In [1]:
import glob
import pandas as pd
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))


#all_algolia_tsv_files = glob.glob("./dumps/queries_1/**/algolia.txt", recursive=True)
#all_es_tsv_files = glob.glob("./dumps/queries_1/**/es.txt", recursive = True)

all_algolia = pd.read_csv('./data/algolia_blr_1week_top50percentile.tsv', sep='\t', header=0)
df2 = pd.read_csv('./data/gt_blr_2week_top50percentile.tsv', sep='\t', header=0)
df2.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#pop_score_df = pd.read_csv('./data/im_popscore_788741_30days_13thapril.tsv', sep='\t', header=0)


,city_id,query,query_rank,query_percentile,productId,productName,q_p_rel,product_rank
0,1,milk,1,0.012726,"""61WDNU5G4B""","""Nandini Pasteurised Toned Milk""",26633,1
1,1,milk,1,0.012726,"""86CIN32V02""","""Heritage Daily Health Toned Milk """,9788,2
2,1,milk,1,0.012726,"""CCX1XWR7GN""","""Nandini Shubam Milk""",9260,3
3,1,milk,1,0.012726,"""13JKWIC8ZS""","""Nandini GoodLife Toned Milk """,7750,4
4,1,milk,1,0.012726,"""MY3HE7Z0AA""","""Nandini GoodLife Toned Milk """,6643,5
...,...,...,...,...,...,...,...,...
22653,1,cerelac,713,0.499887,"""A1ZP6AE3QW""","""Nestle Cerelac Baby Cereal with Milk Rice - F...",211,5
22654,1,cerelac,713,0.499887,"""QJLVO78B3P""","""Nestle Wheat Apple Cherry Cerelac""",206,6
22655,1,cerelac,713,0.499887,"""82YX75EWEI""","""Nestle Cerelac Wheat, Stage 1""",194,7
22656,1,cerelac,713,0.499887,"""WXY0CWXYGH""","""Nestle Cerelac Multigrain Dal Veg - From 12 T...",182,8


In [50]:
len(pop_score_df.loc[pop_score_df['PRODUCT_NAME'] == 'Nandini Curd', 'ORDER_COUNT'])

1

In [40]:
# to read results from multiple folders
es_data = []
algolia_data = []
for file_name in all_es_tsv_files:
    with open(file_name) as file:
        query = file_name.split('/')[3]
        for line in file:
            l=line.split(',')
            es_score = float(l[-1].strip('\n'))
            es_result = ','.join(l[:-1])
            order_counts = pop_score_df.loc[pop_score_df['PRODUCT_NAME'] == es_result, 'ORDER_COUNT']
            if len(order_counts) > 0:
                pop_score = float(order_counts.iloc[0])
            else:
                pop_score = 0
            relevance_score = 0.5*sigmoid(float(es_score)) + 0.0*sigmoid(float(pop_score))
            es_data.append([query, es_result, es_score, pop_score, relevance_score])
print(len(es_data))

for file_name in all_algolia_tsv_files:
    with open(file_name) as file:
        query = file_name.split('/')[3]
        for line in file:
            algolia_result = line.strip('\n')
            algolia_data.append([query, algolia_result, 1])

df_algolia = pd.DataFrame(algolia_data)
df_es = pd.DataFrame(es_data)
#df_algolia.rename(columns = {'0' : 'Query', '1': 'Algolia Result'})



21175


,Query,es,es_score,pop_score,relevance_score,es_sugg_rank
10751,aloo,Potato (Aloo),155.961300,1010.0,0.500000,1.0
10752,aloo,Potato (Aloo) Value Pack,152.216220,267.0,0.500000,2.0
10753,aloo,Potato,136.460390,0.0,0.500000,3.0
10754,aloo,Organic Certified Potato,133.356600,240.0,0.500000,4.0
10755,aloo,Potato,125.397766,0.0,0.500000,5.0
...,...,...,...,...,...,...
4077,water,Lay's Wafer Style Tangy Treat Flavour,0.696575,59.0,0.333714,78.0
4078,water,Dukes Waffy Strawberry Roll,0.695634,16.0,0.333609,79.0
4079,water,Dukes Waffy Orange Wafers,0.675631,84.0,0.331381,80.0
4080,water,Combo (2-Pack) Nestle Kitkat 2 Finger Wafer Bar,0.571102,0.0,0.319509,81.0


In [ ]:

df_algolia.columns = ["Query", "Algolia Results", "algolia_score"]
df_es.columns = ["Query", "Elastic-search results", "es_score", "pop_score", "relevance_score"]

#df_algolia.loc[df_algolia['Query'] == 'aloo']
df_algolia.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_es.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df_algolia["Query"].ffill(inplace=True)
df_es["Query"].ffill(inplace=True)



df1_algolia = df_algolia.rename(columns = {'Algolia Results': 'algolia'})
df1_es = df_es.rename(columns = {'Elastic-search results': 'es'})


df1_algolia['algolia_sugg_rank'] = df1_algolia.groupby('Query')['Query'].rank(method='first')
df1_es['es_sugg_rank'] = df1_es.groupby('Query')['relevance_score'].rank(method = 'first', ascending = False)
df1_es.sort_values(['Query', 'es_sugg_rank'], ascending=[True, True])

In [31]:
len(set(list(df1_es['Query'])))



88

In [32]:
print(len(set(list(df1_es['Query']))))



88


In [41]:
print(len(set(list(df1_es['Query']))))

df1_algolia = df1_algolia[df1_algolia['algolia_sugg_rank'] <= 10]
df1_es = df1_es[df1_es['es_sugg_rank'] <= 10]

print(len(set(list(df1_es['Query']))))

df2['sugg_count'] = df2.groupby('Query')['Query'].transform('count')
df2.sort_values(['Query'])
df2['order_count'].fillna(0, inplace = True)

df2['sugg_rank'] = df2.sort_values(['Query', 'order_count', 'SEARCHES'], ascending=False).groupby(['Query']).cumcount() + 1
df2.sort_values(['Query', 'order_count'], ascending = [1,0])

df2 = df2[df2['sugg_rank'] <= 10]



df2_sel_col = df2[['Query', 'suggestion', 'sugg_rank']].rename(columns = {'suggestion' : 'ground_truth_product', 'sugg_rank': 'gt_product_rank'})



#print(df1_out)
#df1_algolia = df1_out.groupby('Query').agg({'algolia':lambda x: list(x)})
#df1_es = df1_out.groupby('Query').agg({'es':lambda x: list(x)})


88
88


In [42]:
df1_gt = df2_sel_col.groupby('Query').agg({'ground_truth_product':lambda x: list(x)})
df1_es = df1_es.groupby('Query').agg({'es':lambda x: list(x)})
df1_algolia = df1_algolia.groupby('Query').agg({'algolia':lambda x: list(x)})


In [35]:
df1_es.count()

es    88
dtype: int64

In [43]:
dict_algo = df1_algolia.to_dict()#['Algolia Results'].keys()#['paneer']
dict_es = df1_es.to_dict()#['Algolia Results'].keys()#['paneer']
dict_gt = df1_gt.to_dict()#['Algolia Results'].keys()#['paneer']

#print(dict_algo)
#print(dict_es)
#print(dict_gt)

dict_all = dict_algo
dict_all.update(dict_es)
dict_all.update(dict_gt)
#print(dict_all)

avg_recall_es = 0
avg_recall_algolia = 0
count = 0
skip_count = 0
new_recall_dict = {}
avg_ndcg_algolia = 0
avg_ndcg_es = 0
avg_mrr_algolia = 0
avg_mrr_es = 0

for key in dict_algo['algolia']:
    if key not in dict_all['ground_truth_product']:
        skip_count += 1
        continue
    gt_list = [str(p).strip() for p in dict_all['ground_truth_product'][key] if str(p) != 'nan']
    if len(set(gt_list)) == 0:
        skip_count += 1
        continue

    algolia_list = [str(p).strip() for p in dict_all['algolia'][key] if str(p) != 'nan']
    es_list = [str(p).strip() for p in dict_all['es'][key] if str(p) != 'nan']
    #print(key)
    #if key == 'coconut':
        # print(algolia_list)
        # print(gt_list)
        # print(es_list)
        # print(len(set(algolia_list)), len(set(es_list)), len(set(gt_list)))
    recall_algolia = len(set(gt_list).intersection(set(algolia_list)))*100/len(set(gt_list))
    avg_recall_algolia += recall_algolia

    recall_es =  len(set(gt_list).intersection(set(es_list)))*100/len(set(gt_list))
    avg_recall_es += recall_es
    #print(recall_algolia, recall_es)
    if not (recall_algolia == 0 and recall_es == 0):
        count += 1
    else:
        skip_count += 1

    #add info to output dictionary
    new_recall_dict[key] = {'recall_algolia_in_gt_top10': recall_algolia,
                            'recall_es_in_gt_top10': recall_es}

    # ndcg calculation
    relevance_list_algolia = []
    relevance_list_es = []
    if len(algolia_list) == 0:
        print('problem here', key)
    for item in algolia_list:
        if item in gt_list:
            gt_index = gt_list.index(item)
        else:
            gt_index = 10
        if gt_index < 0:
            gt_index = 10
        relevance_list_algolia.append(10-gt_index)

    for item in es_list:
        if item in gt_list:
            gt_index = gt_list.index(item)
        else:
            gt_index = 10

        if gt_index < 0:
            gt_index = 10
        relevance_list_es.append(10-gt_index)

    algolia_dcg = 0
    es_dcg = 0
    algolia_idcg = 0
    es_idcg = 0
    for i in range(len(relevance_list_algolia)):
        algolia_dcg += relevance_list_algolia[i]/math.log2(i+2)
        algolia_idcg += (len(relevance_list_algolia) - i)/math.log2(i+2)

    for i in range(len(relevance_list_es)):
        es_dcg += relevance_list_es[i]/math.log2(i+2)
        es_idcg += (len(relevance_list_es) - i)/math.log2(i+2)


    avg_ndcg_algolia += algolia_dcg/algolia_idcg
    avg_ndcg_es += es_dcg/es_idcg

    #mrr calculation
    first_item = gt_list[0]
    if first_item in algolia_list:
        avg_mrr_algolia += 1/(algolia_list.index(first_item)+1)
    else:
        avg_mrr_algolia += 0

    if first_item in es_list:
        avg_mrr_es += 1/(es_list.index(first_item)+1)
    else:
        avg_mrr_es += 0



avg_recall_algolia = avg_recall_algolia/count
avg_recall_es = avg_recall_es/count
avg_ndcg_algolia = avg_ndcg_algolia/count
avg_ndcg_es = avg_ndcg_es/count
avg_mrr_algolia = avg_mrr_algolia/count
avg_mrr_es = avg_mrr_es/count

print('count, skip_count:', count, skip_count)
print('recall_algolia:', avg_recall_algolia)
print('recall_es:', avg_recall_es)
print('ndcg_algolia:', avg_ndcg_algolia)
print('ndcg_es:', avg_ndcg_es)
print('mrr_algolia:', avg_mrr_algolia)
print('mrr_es:', avg_mrr_es)

new_recall_df = pd.DataFrame.from_dict(new_recall_dict).T
new_recall_df['Query'] = new_recall_df.index
#new_recall_df






count, skip_count: 85 3
recall_algolia: 53.02287581699346
recall_es: 51.643323996265174
ndcg_algolia: 0.5914169570672376
ndcg_es: 0.5778518238464659
mrr_algolia: 0.3422549019607843
mrr_es: 0.36035480859010266


In [ ]:
df1_out.to_csv('./data/im_poc_gt_788741_26apr_2days.tsv', sep = "\t")
